In [13]:
import sys
# getattr(sys.modules[__name__], "Foo")

print(getattr(sys.modules["qcnn.quantum"], "operation"))

<module 'qcnn.quantum.operation' from '/Users/mingyoungjeng/Documents/Spring 2023.nosync/thesis/src/qcnn/quantum/operation/__init__.py'>


In [26]:
import re

test = "qcnn.test"



-1